<a href="https://colab.research.google.com/github/Haarinaath-Y/YouTube-Content-Categorization/blob/main/Youtube_Content_Categorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Youtube Content Categorization

## Init Job

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.4.2/spark-3.4.2-bin-hadoop3.tgz
!tar -xvf spark-3.4.2-bin-hadoop3.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.2-bin-hadoop3"
import findspark
findspark.init()

Import Libraries

In [64]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType,FloatType,DoubleType,DateType

Loading Source Data from Git Repo

In [ ]:
!wget https://raw.githubusercontent.com/Haarinaath-Y/YouTube-Content-Categorization/main/Age.csv
!wget https://raw.githubusercontent.com/Haarinaath-Y/YouTube-Content-Categorization/main/Country.csv
!wget https://raw.githubusercontent.com/Haarinaath-Y/YouTube-Content-Categorization/main/Gender.csv
!wget https://raw.githubusercontent.com/Haarinaath-Y/YouTube-Content-Categorization/main/Person.csv
!wget https://raw.githubusercontent.com/Haarinaath-Y/YouTube-Content-Categorization/main/YTCat.csv
!wget https://raw.githubusercontent.com/Haarinaath-Y/YouTube-Content-Categorization/main/YTCon.csv

Starting the Spark Session

In [123]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

Creating Dataframes from Source Data

In [124]:
df_country = spark.read.csv("Country.csv",header=True,inferSchema=True)
df_country.show()

+----------+------------+------------+---------------+
|Country_ID|Country_code|COUNTRY_NAME|Telephonic_code|
+----------+------------+------------+---------------+
|         1|         IND|       India|             91|
|         2|         USA|         USA|              1|
|         3|         DEU|     Germany|             49|
|         4|         NPL|       Nepal|            977|
|         5|         LKA|   Sri Lanka|             94|
+----------+------------+------------+---------------+



In [125]:
df_ytcat = spark.read.csv("YTCat.csv",header=True,inferSchema=True)
df_ytcat.show()

+--------------+--------------------+--------------------+
|Discription_ID|       Category_Name|       Category_Dsc |
+--------------+--------------------+--------------------+
|             1|                News|Information about...|
|             2|              Sports|Athletic activity...|
|             3|                Kids|A form of activit...|
|             4|      Entertainment |A form of activit...|
|             5|  Fashion and Beauty|The style or styl...|
|             6|Funny Animals and...|Hilarious movemen...|
|             7|            Gameplay|Live streaming of...|
|             8|      Product Review|Unboxing of produ...|
|             9|               Vlogs|A type of content...|
|            10|           Education|Discipline that i...|
+--------------+--------------------+--------------------+



In [126]:
df_ytcon = spark.read.csv("YTCon.csv",header=True,inferSchema=True)
df_ytcon.show()

+--------+--------------------+--------------------+---------+-------+------------+-----------+---------+-------+--------+
|Video_id|         Video_title|          Video_Tags|Person_ID|Min_age|Publish_date|Category_id|    Views|  Likes|Dislikes|
+--------+--------------------+--------------------+---------+-------+------------+-----------+---------+-------+--------+
|       1|         Deadpool 2 |#deadpool #marvel...|       39|    U13|  22-03-2018|          7|  8100000|  11050|    2500|
|       2|  Tom and Jerry show|#WBKids #KidsCart...|       35|    U13|  20-11-2000|          1|300000000|1900900|   50000|
|       3|FIFA Wold Cup 201...|#worldcup #soccer...|      113|    U18|  15-07-2018|          6| 62150000| 510000|   17000|
|       4|Diana's Monochrom...|#influncer #makeu...|      101|    U21|  14-01-2016|          8|    58000|   1800|     360|
|       5|           veg Pulao| #pulao #pulaorec...|      137|    U18|  22-12-2022|          4|       77|     11|       2|
|       6|"Coope

In [127]:
df_ytcon.printSchema()

root
 |-- Video_id: integer (nullable = true)
 |-- Video_title: string (nullable = true)
 |-- Video_Tags: string (nullable = true)
 |-- Person_ID: integer (nullable = true)
 |-- Min_age: string (nullable = true)
 |-- Publish_date: string (nullable = true)
 |-- Category_id: integer (nullable = true)
 |-- Views: integer (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- Dislikes: integer (nullable = true)



In [141]:
df_person = spark.read.csv("Person.csv",header=True,inferSchema=True)
df_person.show()

+---------+----------+------------+----------+-----------+------------+------------+--------------------+--------------------+------------+
|Person_ID|       DOB|Initial_Name|First_Name|  Last_Name|Country_Name|Phone_number|               Email|        Account_name|Subscription|
+---------+----------+------------+----------+-----------+------------+------------+--------------------+--------------------+------------+
|        1|1999-03-21|          Ms|    Sakshi|      Gupta|       India|  9876532945|       sakshig@gmail|  Sakshiii Gupta9900|           Y|
|        2|1993-04-11|          Mr|      Alan|     Cooper|         USA|240-219-0550|  CooperAlan@hotmail|             CooperA|           N|
|        3|1990-11-08|          Mr|    Yohani|         De|   Sri Lanka|  8797321854|   yoohani@yahoo.com|         Telly Music|           Y|
|        4|2000-04-06|         Mrs|      Otto|      Smith|     Germany|  7840096538|smithbakery@gmail...|      Smith's Bakery|           N|
|        5|1979-08-1

In [142]:
df_person.printSchema()

root
 |-- Person_ID: integer (nullable = true)
 |-- DOB: date (nullable = true)
 |-- Initial_Name: string (nullable = true)
 |-- First_Name: string (nullable = true)
 |-- Last_Name: string (nullable = true)
 |-- Country_Name: string (nullable = true)
 |-- Phone_number: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Account_name: string (nullable = true)
 |-- Subscription: string (nullable = true)



Removed Special Characters on Initial Name

In [145]:
df_person = df_person.withColumn('Initial_Name',regexp_replace(col('Initial_Name'),"[^0-9a-zA-Z_\-]+", ""))

Gender Definition using Initial Name

In [146]:
df_person = df_person.withColumn('Gender',
                                 when ((df_person['Initial_Name'] == 'Ms'),lit('F')).
                                 when ((df_person['Initial_Name'] == 'Mrs'),lit('F')).
                                 when ((df_person['Initial_Name'] == 'Mr'),lit('M')).
                                 otherwise(lit('UK')))

Calculating Age by subtracting DOB with current date

In [147]:
df_person = df_person.withColumn('Age',round(months_between(current_date(),col("DOB"))/lit(12),0).cast(IntegerType()))

Concatenating First Name and Last Name

In [148]:
df_person = df_person.withColumn('Person_Name',concat_ws(" ", col('First_Name'),col('Last_Name')))

Removing special characters in Phone Number

In [149]:
df_person = df_person.withColumn('Phone_number',regexp_replace(col('Phone_number'),"[^0-9]",""))

Dropping Null Values in Country Name column

In [150]:
df_person = df_person.na.drop(subset=['Country_Name'])

Creating View Tables for Data Transformation for Youtuber's data


In [151]:
person = df_person.createOrReplaceTempView('df_person')
country = df_country.createOrReplaceTempView('df_country')

Phone Number Data Transformation by joining country and person tables

In [152]:
user = spark.sql("select Person_ID,Person_Name,Gender,Age,concat('+',c.Telephonic_code,p.Phone_number) as Phone_No, p.Country_Name from df_person p join df_country c on p.Country_Name = c.COUNTRY_NAME")

In [153]:
user.printSchema()

root
 |-- Person_ID: integer (nullable = true)
 |-- Person_Name: string (nullable = false)
 |-- Gender: string (nullable = false)
 |-- Age: integer (nullable = true)
 |-- Phone_No: string (nullable = true)
 |-- Country_Name: string (nullable = true)



In [154]:
user.show()

+---------+------------------+------+---+--------------+------------+
|Person_ID|       Person_Name|Gender|Age|      Phone_No|Country_Name|
+---------+------------------+------+---+--------------+------------+
|        1|      Sakshi Gupta|     F| 25| +919876532945|       India|
|        2|       Alan Cooper|     M| 31|  +12402190550|         USA|
|        3|         Yohani De|     M| 33| +948797321854|   Sri Lanka|
|        4|        Otto Smith|     F| 24| +497840096538|     Germany|
|        5|           Wang Li|     F| 45| +917781435789|       India|
|        6|    Neeru Srikanth|     F| 39| +915673831214|       India|
|        7|    Shivin Ganesan|     F| 21| +918769452345|       India|
|        8|    Hannah Pauline|     M| 25|  +19562604620|         USA|
|        9|  Lora Santhakumar|     F| 29| +917864532103|       India|
|       10|Sriram  Saptarishi|     M| 23| +919840567898|       India|
|       11|     Gabriel Lucas|     F| 24|  +17075379647|         USA|
|       12|      San

Calculating Engagement Metrics using Likes and Dislikes

In [155]:
df_ytcon = df_ytcon.withColumn('Engagement',df_ytcon['Likes']-df_ytcon['Dislikes'])

Dropping null values

In [156]:
df_ytcon = df_ytcon.dropna('any')

Converting the date from String to Date format

In [157]:
df_ytcon = df_ytcon.withColumn("Publish_date",to_date(col("Publish_date"),"dd-mm-yyyy").alias("date"))

Creating View tables for Data Transformation for Youtube Data

In [158]:
content = df_ytcon.createOrReplaceTempView('df_ytcon')
category = df_ytcat.createOrReplaceTempView('df_ytcat')

Youtube Data Transformation

In [159]:
video = spark.sql("select Video_id, v.Person_ID, Video_title, cat.Category_Name as Category, Min_age as Age_Gate, p.Country_Name,  v.Publish_date,Engagement, Views as Total_Views, CASE WHEN Views <= 4000000 THEN 'Low' WHEN Views > 4000000 AND Views <= 8000000 THEN 'Medium' ELSE 'High' END as Trend_Level from df_ytcon v join df_person p on v.Person_ID = p.Person_ID join df_ytcat cat on v.category_ID = cat.Discription_ID")

In [160]:
video.show(truncate=False)

+--------+---------+--------------------------------------------------------------------------------------+-----------------------+--------+------------+------------+----------+-----------+-----------+
|Video_id|Person_ID|Video_title                                                                           |Category               |Age_Gate|Country_Name|Publish_date|Engagement|Total_Views|Trend_Level|
+--------+---------+--------------------------------------------------------------------------------------+-----------------------+--------+------------+------------+----------+-----------+-----------+
|2       |35       |Tom and Jerry show                                                                    |News                   |U13     |Germany     |2000-01-20  |1850900   |300000000  |High       |
|3       |113      |FIFA Wold Cup 2018 Highights                                                          |Funny Animals and Birds|U18     |Germany     |2018-01-15  |493000    |62150000   |Hig

In [161]:
video.printSchema()

root
 |-- Video_id: integer (nullable = true)
 |-- Person_ID: integer (nullable = true)
 |-- Video_title: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Age_Gate: string (nullable = true)
 |-- Country_Name: string (nullable = true)
 |-- Publish_date: date (nullable = true)
 |-- Engagement: integer (nullable = true)
 |-- Total_Views: integer (nullable = true)
 |-- Trend_Level: string (nullable = false)



## Creating View Tables for the final output data

In [162]:
video.createOrReplaceTempView('video')
user.createOrReplaceTempView('user')

# Business Insights

No of videos based on Trend level

In [163]:
spark.sql("SELECT Trend_Level, count(Video_ID) as Count from video group by 1 order by 2 desc;").show()

+-----------+-----+
|Trend_Level|Count|
+-----------+-----+
|        Low|   59|
|     Medium|   59|
|       High|   30|
+-----------+-----+



No of videos based on Age_Gate

In [164]:
spark.sql("SELECT Age_Gate, count(Video_ID) as Count from video group by Age_Gate order by 2 desc;").show()

+--------+-----+
|Age_Gate|Count|
+--------+-----+
|     U13|   80|
|     U18|   43|
|     U21|   18|
|     T21|    7|
+--------+-----+



Engagement on each category

In [165]:
spark.sql("SELECT Category, sum(Engagement) as Total_Engagement from video group by 1 order by 2 desc;").show()

+--------------------+----------------+
|            Category|Total_Engagement|
+--------------------+----------------+
|                News|         2060278|
|Funny Animals and...|          768037|
|  Fashion and Beauty|          426680|
|                Kids|          350137|
|            Gameplay|          310155|
|              Sports|          243551|
|               Vlogs|          203234|
|           Education|          125834|
|      Entertainment |          124742|
|      Product Review|           88420|
+--------------------+----------------+



Top 10 Youtubers posted the highest no of videos

In [166]:
spark.sql("SELECT u.Person_ID, u.Person_Name, count(v.Video_title) as Count from video v join user u on v.Person_ID = u.Person_ID group by 1,2 order by 3 desc limit 10").show()

+---------+----------------+-----+
|Person_ID|     Person_Name|Count|
+---------+----------------+-----+
|      188|  Victor Cecilia|    4|
|      195| Fernando Kaylee|    4|
|       99|Griffin Arabella|    4|
|      148|  Messiah Xander|    3|
|       78| Alexandra Amaya|    3|
|       88|    Sage Brynlee|    3|
|      178|Crisp William E.|    3|
|       80|  Jasmine Elango|    3|
|       62|    Karan Vathan|    3|
|       63|     Zara Zhahir|    3|
+---------+----------------+-----+



No of videos posted from each Country

In [167]:
spark.sql("SELECT u.Country_Name, count(v.Video_title) from video v join user u on v.Person_ID = u.Person_ID group by 1 order by 2 desc").show()

+------------+------------------+
|Country_Name|count(Video_title)|
+------------+------------------+
|         USA|                49|
|       India|                45|
|     Germany|                33|
|   Sri Lanka|                13|
|       Nepal|                 8|
+------------+------------------+



Average Views Per Day based on Publish date

In [168]:
spark.sql("SELECT Publish_date, CASE WHEN datediff(current_date(), v.Publish_date) = 0 THEN v.Total_Views ELSE round(v.Total_Views / (DATEDIFF(CURRENT_DATE, v.Publish_date) + 1),2) END AS average_views_per_day from video v join user u on v.Person_ID = u.Person_ID order by 2 desc").show()

+------------+---------------------+
|Publish_date|average_views_per_day|
+------------+---------------------+
|  2000-01-20|             33978.93|
|  2018-01-15|             27512.17|
|  2022-01-14|             12131.53|
|  2021-01-05|              8332.42|
|  2021-01-21|              6546.85|
|  2022-01-05|              6209.92|
|  2020-01-24|              6199.54|
|  2022-01-05|              5621.49|
|  2022-01-19|               5375.3|
|  2020-01-27|              5154.44|
|  2020-01-09|               4764.6|
|  2021-01-31|              4644.52|
|  2019-01-10|              4374.15|
|  2019-01-24|              4334.34|
|  2018-01-11|              4298.58|
|  2018-01-23|              3970.85|
|  2019-01-05|              3904.39|
|  2018-01-14|              3902.06|
|  2020-01-17|              3898.99|
|  2022-01-28|              3753.05|
+------------+---------------------+
only showing top 20 rows

